In [2]:
%load_ext autoreload
%autoreload 2
from geolifeclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()
metadata = spark.read.parquet(
    "../../data/geolifeclef-2024/data/processed/metadata_clean/v1"
)
metadata.printSchema()
metadata.show()

24/05/20 16:13:27 WARN Utils: Your hostname, daphne-major resolves to a loopback address: 127.0.1.1; using 172.28.199.217 instead (on interface eth0)
24/05/20 16:13:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/20 16:13:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/20 16:13:29 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


root
 |-- dataset: string (nullable = true)
 |-- surveyId: integer (nullable = true)
 |-- lat_proj: double (nullable = true)
 |-- lon_proj: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- geoUncertaintyInM: double (nullable = true)
 |-- speciesId: double (nullable = true)



+--------+--------+-------------------+--------------------+---------+---------+----+-----------------+---------+
| dataset|surveyId|           lat_proj|            lon_proj|      lat|      lon|year|geoUncertaintyInM|speciesId|
+--------+--------+-------------------+--------------------+---------+---------+----+-----------------+---------+
|pa_train| 1154667|-1532011.0411057267| 1.691006027258277E7| 57.41102|  10.1656|2018|             10.0|   5928.0|
|      po|  328943|-1238969.8380109437|1.6157490707117997E7| 52.66047|  19.0257|2021|              4.0|   8654.0|
|      po|  272993| -3131000.345551669|1.5794519571905952E7| 42.70678| 0.578367|2021|             7.67|  10929.0|
|      po|   42950| -2693959.769707624|1.6015928734426929E7| 46.04146| 3.254066|2020|            21.44|   6837.0|
|      po|   32868| -2341953.455934548|1.6504383916338675E7| 50.90031|  3.52936|2020|             25.0|   2112.0|
|pa_train| 1655162|-2702929.1847960646|1.6298793816959836E7| 47.87137|  1.32333|2019|   

In [3]:
metadata.groupBy("dataset").count().show()

+--------+-------+
| dataset|  count|
+--------+-------+
|      po|5079797|
|pa_train|1483637|
| pa_test|   4716|
+--------+-------+



In [4]:
metadata.select("speciesId").distinct().count()

10359

In [5]:
metadata.where("dataset='pa_train'").select("speciesId").distinct().count()

5016

In [6]:
(
    metadata.groupBy("speciesId")
    .agg(F.count("surveyId").alias("n"))
    .groupBy("n")
    .agg(F.count("*").alias("freq"))
    .orderBy("n")
).show()

+---+----+
|  n|freq|
+---+----+
|  1|1526|
|  2| 795|
|  3| 478|
|  4| 363|
|  5| 267|
|  6| 237|
|  7| 192|
|  8| 172|
|  9| 143|
| 10| 123|
| 11| 130|
| 12| 112|
| 13|  95|
| 14| 100|
| 15|  86|
| 16|  85|
| 17|  93|
| 18|  87|
| 19|  68|
| 20|  68|
+---+----+
only showing top 20 rows



In [7]:
metadata.select("speciesId").describe().show()

+-------+-----------------+
|summary|        speciesId|
+-------+-----------------+
|  count|          6563434|
|   mean|5607.773993156631|
| stddev| 3296.91880881717|
|    min|              0.0|
|    max|          11254.0|
+-------+-----------------+



In [8]:
metadata.select(F.max("speciesId")).show()

+--------------+
|max(speciesId)|
+--------------+
|       11254.0|
+--------------+



In [9]:
metadata.groupBy("dataset").agg(F.countDistinct("surveyId")).show()

+--------+------------------------+
| dataset|count(DISTINCT surveyId)|
+--------+------------------------+
|      po|                 3845533|
|pa_train|                   88987|
| pa_test|                    4716|
+--------+------------------------+



In [10]:
for i in range(0, 10, 2):
    print("num_species", i)
    (
        metadata.groupBy("dataset", "surveyId")
        .agg(F.countDistinct("speciesId").alias("n_species"))
        .where(f"n_species > {i}")
        .groupBy("dataset")
        .count()
    ).show()

num_species 0


+--------+-------+
| dataset|  count|
+--------+-------+
|      po|3845533|
|pa_train|  88987|
+--------+-------+

num_species 2


+--------+------+
| dataset| count|
+--------+------+
|      po|170215|
|pa_train| 87440|
+--------+------+

num_species 4


+--------+------+
| dataset| count|
+--------+------+
|      po|104124|
|pa_train| 84061|
+--------+------+

num_species 6


+--------+-----+
| dataset|count|
+--------+-----+
|      po|66182|
|pa_train|78411|
+--------+-----+

num_species 8


+--------+-----+
| dataset|count|
+--------+-----+
|      po|42094|
|pa_train|70553|
+--------+-----+



In [11]:
subset = metadata.join(
    metadata.groupBy("dataset", "surveyId")
    .agg(F.countDistinct("speciesId").alias("n_species"))
    .where("n_species > 5")
    .select("surveyId"),
    on="surveyId",
)
subset.select(F.countDistinct("speciesId")).show()
(
    subset.groupBy("surveyId")
    .agg(F.countDistinct("speciesId").alias("n_species"))
    .describe()
).show()

+-------------------------+
|count(DISTINCT speciesId)|
+-------------------------+
|                     6443|
+-------------------------+



+-------+------------------+------------------+
|summary|          surveyId|         n_species|
+-------+------------------+------------------+
|  count|            154606|            154606|
|   mean|1961119.9113294438|14.094433592486709|
| stddev|1132436.1905773734| 8.565677476416594|
|    min|                26|                 6|
|    max|           3919655|               163|
+-------+------------------+------------------+



In [12]:
metadata.groupBy("dataset").agg(F.countDistinct("surveyId")).show()

+--------+------------------------+
| dataset|count(DISTINCT surveyId)|
+--------+------------------------+
|      po|                 3845533|
|pa_train|                   88987|
| pa_test|                    4716|
+--------+------------------------+

